# LangGraph State 관리 튜토리얼

이 튜토리얼에서는 LangGraph에서 **State가 어떻게 관리되고 실행되는지** 구체적으로 알아봅니다.

## 핵심 개념

| 개념 | 설명 |
|------|------|
| **State** | 그래프 전체에서 공유되는 상태 객체 (TypedDict) |
| **Node** | 상태를 받아서 변환하는 함수 |
| **Reducer** | 여러 노드의 출력을 어떻게 병합할지 정의 |
| **Checkpointer** | 각 노드 실행 후 상태 스냅샷을 저장 |
| **MemorySaver** | 인메모리 Checkpointer 구현체 |
| **thread_id** | 독립적인 대화/세션 식별자 |

## 그래프 구조

```
START → [counter] → [processor] → [logger] → END
           ↓             ↓            ↓
       count += 1    결과 생성     로그 기록
```

## 1. 환경 설정

In [ ]:
from typing import Annotated, TypedDict, List
from datetime import datetime

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

print("LangGraph 임포트 완료!")

## 2. State 정의 이해하기

### State란?
- 그래프의 모든 노드가 **공유하는 데이터 구조**
- `TypedDict`로 정의하여 타입 안전성 확보
- 각 노드는 State를 입력받고, **변경된 부분만** 반환

### Reducer란?
- 노드가 반환한 값을 기존 State에 **어떻게 병합할지** 정의
- 기본 동작: 단순 덮어쓰기 (overwrite)
- `Annotated[List, operator.add]`: 리스트에 append

In [ ]:
import operator


class GraphState(TypedDict):
    """그래프의 상태를 정의합니다."""
    
    # 기본 필드: 단순 덮어쓰기 (overwrite)
    user_input: str
    
    # 카운터: 덮어쓰기
    count: int
    
    # 처리 결과: 덮어쓰기
    result: str
    
    # 로그: Reducer 사용 - 리스트에 누적 (append)
    logs: Annotated[List[str], operator.add]


print("GraphState 정의 완료!")
print("\n필드 구성:")
print("  - user_input: str (덮어쓰기)")
print("  - count: int (덮어쓰기)")
print("  - result: str (덮어쓰기)")
print("  - logs: List[str] (누적 - operator.add Reducer)")

## 3. 노드 정의 - 상태 업데이트 방식

### 핵심 포인트
1. 노드는 **전체 State**를 입력으로 받음
2. 노드는 **변경된 필드만** 반환하면 됨 (부분 업데이트)
3. LangGraph가 자동으로 **기존 State와 병합**

In [ ]:
def counter_node(state: GraphState) -> GraphState:
    """
    [Node 1: Counter]
    카운트를 1 증가시킵니다.
    
    입력: 전체 State
    출력: 변경된 필드만 반환 (count, logs)
    """
    current_count = state.get("count", 0)
    new_count = current_count + 1
    
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_msg = f"[{timestamp}] counter_node: count {current_count} → {new_count}"
    
    print(f"  🔢 counter_node 실행: {current_count} → {new_count}")
    
    # 변경된 필드만 반환!
    return {
        "count": new_count,
        "logs": [log_msg]  # Reducer가 기존 리스트에 append
    }


def processor_node(state: GraphState) -> GraphState:
    """
    [Node 2: Processor]
    사용자 입력을 처리하여 결과를 생성합니다.
    """
    user_input = state.get("user_input", "")
    count = state.get("count", 0)
    
    # 간단한 처리 로직
    result = f"처리 완료: '{user_input}' (실행 #{count})"
    
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_msg = f"[{timestamp}] processor_node: 입력 '{user_input}' 처리"
    
    print(f"  ⚙️ processor_node 실행: '{user_input}' 처리")
    
    return {
        "result": result,
        "logs": [log_msg]
    }


def logger_node(state: GraphState) -> GraphState:
    """
    [Node 3: Logger]
    최종 로그를 기록합니다.
    """
    result = state.get("result", "")
    
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_msg = f"[{timestamp}] logger_node: 완료 - {result}"
    
    print(f"  📝 logger_node 실행: 로그 기록")
    
    return {
        "logs": [log_msg]
    }


print("노드 함수 정의 완료!")
print("\n각 노드의 역할:")
print("  1. counter_node: count += 1")
print("  2. processor_node: 입력 처리 → result 생성")
print("  3. logger_node: 완료 로그 기록")

## 4. MemorySaver로 그래프 구성

### MemorySaver란?
- **인메모리 Checkpointer** 구현체
- 각 노드 실행 후 **State 스냅샷**을 저장
- `thread_id`별로 독립적인 히스토리 관리
- 개발/테스트용으로 적합 (프로덕션에서는 SqliteSaver, PostgresSaver 등 사용)

In [ ]:
# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("counter", counter_node)
workflow.add_node("processor", processor_node)
workflow.add_node("logger", logger_node)

# 엣지 정의 (순차 실행)
workflow.add_edge(START, "counter")
workflow.add_edge("counter", "processor")
workflow.add_edge("processor", "logger")
workflow.add_edge("logger", END)

# MemorySaver 생성
memory = MemorySaver()

# 그래프 컴파일 (checkpointer 연결!)
graph = workflow.compile(checkpointer=memory)

print("✅ 그래프 컴파일 완료!")
print("\n📦 MemorySaver 연결됨 - 모든 상태 변화가 저장됩니다.")

In [ ]:
# 그래프 시각화
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 실패: {e}")
    print("\n그래프 구조:")
    print("START → counter → processor → logger → END")

## 5. 그래프 실행 & 상태 변화 관찰

### thread_id란?
- 대화/세션을 식별하는 고유 ID
- 같은 `thread_id` → 상태가 **연속**됨
- 다른 `thread_id` → **새로운** 대화 시작

In [ ]:
# 첫 번째 실행
print("=" * 60)
print("🚀 첫 번째 실행")
print("=" * 60)

config = {"configurable": {"thread_id": "session-1"}}

result1 = graph.invoke(
    {"user_input": "안녕하세요"},
    config=config
)

print("\n📊 최종 State:")
for key, value in result1.items():
    print(f"  {key}: {value}")

## 6. get_state()로 현재 상태 조회

`get_state()`는 특정 thread의 **현재(최신) 상태**를 조회합니다.

In [ ]:
# 현재 상태 조회
current_state = graph.get_state(config)

print("📍 현재 상태 (get_state)")
print("=" * 40)
print(f"\n값 (values):")
for key, value in current_state.values.items():
    print(f"  {key}: {value}")

print(f"\n메타데이터:")
print(f"  next: {current_state.next}")  # 다음 실행할 노드
print(f"  config: {current_state.config}")

## 7. get_state_history()로 전체 히스토리 조회 ⭐

이것이 핵심입니다! `get_state_history()`는 **모든 상태 변화 과정**을 보여줍니다.

In [ ]:
print("📜 상태 히스토리 (get_state_history)")
print("=" * 60)
print("\n각 노드 실행 후 저장된 State 스냅샷:\n")

history = list(graph.get_state_history(config))

# 역순으로 저장되어 있으므로 뒤집어서 시간순으로 표시
for i, state_snapshot in enumerate(reversed(history)):
    print(f"{'─' * 50}")
    print(f"📸 스냅샷 #{i}")
    
    # 어떤 노드에서 저장되었는지
    metadata = state_snapshot.metadata
    source = metadata.get("source", "unknown")
    step = metadata.get("step", -1)
    
    print(f"   단계: {step}")
    print(f"   소스: {source}")
    print(f"   다음 노드: {state_snapshot.next}")
    print(f"   상태값:")
    
    for key, value in state_snapshot.values.items():
        if key == "logs":
            print(f"      {key}: [{len(value)} 개의 로그]")
        else:
            print(f"      {key}: {value}")

## 8. 상태 변화 시각화

각 노드를 통과하면서 State가 어떻게 변화하는지 표로 정리합니다.

In [ ]:
def visualize_state_changes(config):
    """상태 변화를 시각적으로 보여줍니다."""
    history = list(graph.get_state_history(config))
    
    print("\n🔄 State 변화 흐름")
    print("=" * 70)
    print(f"{'단계':<6} {'노드':<15} {'count':<8} {'result':<25} {'logs 수':<8}")
    print("─" * 70)
    
    for state_snapshot in reversed(history):
        step = state_snapshot.metadata.get("step", -1)
        
        # 다음에 실행될 노드로 현재 위치 파악
        next_nodes = state_snapshot.next
        if next_nodes:
            if isinstance(next_nodes, tuple):
                current = f"→ {next_nodes[0]}"
            else:
                current = f"→ {next_nodes}"
        else:
            current = "완료 (END)"
        
        values = state_snapshot.values
        count = values.get("count", "-")
        result = values.get("result", "-")
        if result and len(str(result)) > 22:
            result = str(result)[:22] + "..."
        logs_count = len(values.get("logs", []))
        
        print(f"{step:<6} {current:<15} {str(count):<8} {str(result):<25} {logs_count:<8}")


visualize_state_changes(config)

## 9. 대화 이어가기 (Thread 연속성) ⭐

**같은 thread_id**로 다시 호출하면, 이전 상태가 유지됩니다!

In [ ]:
# 같은 thread_id로 두 번째 실행
print("=" * 60)
print("🚀 두 번째 실행 (같은 thread_id: session-1)")
print("=" * 60)

result2 = graph.invoke(
    {"user_input": "두 번째 메시지입니다"},
    config=config  # 같은 config (thread_id: session-1)
)

print("\n📊 최종 State:")
print(f"  count: {result2['count']}  ← 이전 값에서 +1 되었음!")
print(f"  result: {result2['result']}")
print(f"  logs: {len(result2['logs'])}개  ← 이전 로그에 누적됨!")

In [ ]:
# 세 번째 실행
print("=" * 60)
print("🚀 세 번째 실행 (같은 thread_id: session-1)")
print("=" * 60)

result3 = graph.invoke(
    {"user_input": "세 번째!"},
    config=config
)

print("\n📊 최종 State:")
print(f"  count: {result3['count']}  ← 계속 누적!")
print(f"  logs 수: {len(result3['logs'])}개")

In [ ]:
# 전체 로그 확인
print("📋 전체 로그 히스토리")
print("=" * 60)
for i, log in enumerate(result3["logs"], 1):
    print(f"{i:2}. {log}")

## 10. 새로운 Thread 시작

**다른 thread_id**를 사용하면 완전히 새로운 대화가 시작됩니다.

In [ ]:
# 새로운 thread_id로 실행
print("=" * 60)
print("🆕 새로운 Thread 시작 (thread_id: session-2)")
print("=" * 60)

config2 = {"configurable": {"thread_id": "session-2"}}

result_new = graph.invoke(
    {"user_input": "새로운 대화입니다"},
    config=config2
)

print("\n📊 최종 State:")
print(f"  count: {result_new['count']}  ← 1부터 다시 시작!")
print(f"  logs 수: {len(result_new['logs'])}개  ← 새로운 로그!")

In [ ]:
# 두 세션 비교
print("\n🔍 두 Thread 비교")
print("=" * 40)

state1 = graph.get_state(config)
state2 = graph.get_state(config2)

print(f"\nsession-1:")
print(f"  count: {state1.values.get('count')}")
print(f"  logs: {len(state1.values.get('logs', []))}개")

print(f"\nsession-2:")
print(f"  count: {state2.values.get('count')}")
print(f"  logs: {len(state2.values.get('logs', []))}개")

## 11. 특정 시점으로 상태 복원 (Time Travel)

`get_state_history()`에서 얻은 checkpoint를 사용해 **과거 상태로 되돌릴** 수 있습니다.

In [ ]:
# session-1의 히스토리 확인
print("📜 session-1 히스토리에서 복원 가능한 지점들")
print("=" * 60)

history = list(graph.get_state_history(config))

for i, snapshot in enumerate(history[:5]):  # 최근 5개만
    step = snapshot.metadata.get("step", -1)
    count = snapshot.values.get("count", "-")
    checkpoint_id = snapshot.config["configurable"].get("checkpoint_id", "N/A")
    
    print(f"\n[{i}] Step {step}")
    print(f"    count: {count}")
    print(f"    checkpoint_id: {checkpoint_id[:20]}...")

In [ ]:
# 특정 checkpoint에서 이어서 실행
print("\n⏪ 과거 상태에서 이어서 실행하기")
print("=" * 60)

# 첫 번째 실행 직후 상태를 찾기 (step이 낮은 것)
old_snapshot = history[-4]  # 오래된 스냅샷 선택

print(f"선택한 스냅샷:")
print(f"  step: {old_snapshot.metadata.get('step')}")
print(f"  count: {old_snapshot.values.get('count')}")

# 해당 checkpoint에서 이어서 실행
restored_config = old_snapshot.config

print(f"\n이 시점에서 새로운 입력으로 실행...")
result_restored = graph.invoke(
    {"user_input": "과거에서 분기된 메시지"},
    config=restored_config
)

print(f"\n결과:")
print(f"  count: {result_restored['count']}")
print(f"  result: {result_restored['result']}")

## 12. 요약

### State 관리 핵심 개념

```
┌─────────────────────────────────────────────────────────────┐
│                        LangGraph                            │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   State (TypedDict)                                         │
│   ┌─────────────────────────────────────────────┐          │
│   │ user_input: str                             │          │
│   │ count: int                                  │          │
│   │ result: str                                 │          │
│   │ logs: List[str]  ← Reducer로 누적           │          │
│   └─────────────────────────────────────────────┘          │
│                         │                                   │
│                         ▼                                   │
│   ┌─────────┐    ┌─────────────┐    ┌────────┐             │
│   │ counter │───▶│  processor  │───▶│ logger │             │
│   └─────────┘    └─────────────┘    └────────┘             │
│        │               │                 │                  │
│        ▼               ▼                 ▼                  │
│   {count: +1}    {result: ...}     {logs: [...]}           │
│                                                             │
│                  MemorySaver (Checkpointer)                 │
│   ┌─────────────────────────────────────────────┐          │
│   │ thread_id: "session-1"                      │          │
│   │   └─ snapshot #0 → #1 → #2 → #3 → ...      │          │
│   │ thread_id: "session-2"                      │          │
│   │   └─ snapshot #0 → #1 → ...                │          │
│   └─────────────────────────────────────────────┘          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 핵심 메서드

| 메서드 | 설명 |
|--------|------|
| `graph.invoke(input, config)` | 그래프 실행 |
| `graph.get_state(config)` | 현재 상태 조회 |
| `graph.get_state_history(config)` | 전체 상태 히스토리 조회 |
| `graph.update_state(config, values)` | 상태 수동 업데이트 |

### 기억할 점

1. **노드는 부분 업데이트** - 변경된 필드만 반환하면 자동 병합
2. **Reducer로 누적 가능** - `Annotated[List, operator.add]`
3. **thread_id로 세션 분리** - 독립적인 대화 관리
4. **Checkpointer로 히스토리 저장** - 모든 상태 변화 추적 가능
5. **Time Travel 가능** - 과거 상태에서 분기하여 실행 가능

In [ ]:
print("🎉 튜토리얼 완료!")
print("\n이제 LangGraph의 State 관리 방식을 이해하셨습니다.")
print("\n다음 단계:")
print("  1. SqliteSaver로 영구 저장소 사용해보기")
print("  2. 조건부 엣지로 분기 처리 추가하기")
print("  3. Human-in-the-loop 패턴 구현하기")